In [8]:
import os
import numpy as np
import pandas as pd

In [9]:
from lcmcmc.utils import get_data_dir_path

In [10]:
#Load the data for the header file and photometric file.
path_perfect_sims_phot = "/sps/lsst/users/bbiswas/data/kilonova_datasets/RESSPECT/RESSPECT_PERFECT_LIGHTCURVE_WITH_KN.csv"
path_perfect_sims_head = "/sps/lsst/users/bbiswas/data/kilonova_datasets/RESSPECT/RESSPECT_PERFECT_HEAD_WITH_KN.csv"

df_head = pd.read_csv(path_perfect_sims_head)
df_phot = pd.read_csv(path_perfect_sims_phot)

In [11]:
#extract the objects as in the KN paper 

selected_ids = np.load(os.path.join(get_data_dir_path(), "selected_ids.npy"))

df_head = df_head[df_head["SNID"].isin(list(selected_ids))]
df_phot = df_phot[df_phot["SNID"].isin(list(selected_ids))]

In [12]:
# Use only g and r band

mask = (df_phot['FLT'] == 'g') | (df_phot['FLT'] == 'r')
df_phot = df_phot[mask]

# Preprocessing

In [13]:
from lcmcmc.preprocessing import add_object_band_index, preprocess_SNANA, extract_subsample

In [18]:
df_head, df_phot=extract_subsample(df_head=df_head, df_phot=df_phot, event_type="random", num_sample=50)

In [19]:
df_phot

,SNID,MJD,FLT,FLUXCAL,FLUXCALERR,SNR,object_index,band_index
76207,6878.0,53060.0,g,5.518656,0.160117,34.466423,0,1
76208,6878.0,53060.0,r,7.757073,0.166961,46.460510,0,0
76213,6878.0,53062.0,g,5.938479,0.161423,36.788414,0,1
76214,6878.0,53062.0,r,8.511786,0.169206,50.304390,0,0
76219,6878.0,53064.0,g,6.344034,0.162674,38.998474,0,1
...,...,...,...,...,...,...,...,...
5493859,10984.0,53412.0,r,-0.001926,0.004136,-0.465681,49,0
5493860,10984.0,53414.0,r,0.000009,0.002776,0.003275,49,0
5493861,10984.0,53416.0,r,0.006944,0.006693,1.037394,49,0
5493862,10984.0,53418.0,r,0.007167,0.006646,1.078364,49,0


In [20]:
df_phot = add_object_band_index(df_phot)

In [21]:
# get only 20 days of data before and after maximum 

new_object_df = preprocess_SNANA(df_head=df_head, df_phot=df_phot)
new_object_df = new_object_df[((new_object_df["time"]>-20) & (new_object_df["time"]<20))]
new_object_df

,SNID,time,flux,fluxerr,object_index,band_index,norm_factor
50,6878.0,-18.0,0.452737,0.005925,0,1,31.239656
51,6878.0,-18.0,0.847832,0.006910,0,0,31.239656
52,6878.0,-16.0,0.458199,0.005940,0,1,31.239656
53,6878.0,-16.0,0.859261,0.006937,0,0,31.239656
54,6878.0,-14.0,0.459700,0.005944,0,1,31.239656
...,...,...,...,...,...,...,...
8805,10984.0,10.0,0.009542,0.000312,49,0,8.938220
8806,10984.0,12.0,0.006537,0.000498,49,0,8.938220
8807,10984.0,14.0,0.005009,0.000264,49,0,8.938220
8808,10984.0,16.0,0.004032,0.000836,49,0,8.938220


In [22]:
data_dir = get_data_dir_path()
new_object_df.to_csv(os.path.join(data_dir, "perfect_sims.csv"))